## Problem 1: SADC Tourism Management System

This model captures how a SADC-wide tour operator would plan and sell itineraries across multiple countries with seasonal pricing and limited capacity. I build a small set of cooperating classes: Country (with a month-to-season calendar and price multipliers), an abstract Experience with concrete types like Safari, Beach, Cultural and Adventure (each responsible for its own pricing and per-date availability), Package and PackageItem (to assemble dated activities into an itinerary), TouristGroup (budget, size and preferences to validate against), and a Booking/BookingManager pair (to preview costs, reserve capacity and enforce rules). Together, these components price items by date and country, detect when a package is cross-border, check that a group’s budget and interests are respected, and then confirm or reject a booking based on real availability—demonstrating clean encapsulation, sensible inheritance and composition, and explicit business-rule validation.

In [15]:
# ===========================[ Imports & Exceptions ]================
from __future__ import annotations

from dataclasses import dataclass, field
from datetime import date, datetime
from enum import Enum
from typing import Dict, List, Optional
from abc import ABC, abstractmethod

# Domain-specific exceptions
class BookingError(Exception):
    pass

class AvailabilityError(Exception):
    pass

class ValidationError(Exception):
    pass


In [16]:
# ===========================[ Country & Season ]====================
class Season(Enum):
    LOW = "LOW"
    SHOULDER = "SHOULDER"
    HIGH = "HIGH"
    PEAK = "PEAK"

@dataclass(frozen=True)
class Country:
    name: str
    currency: str = "ZAR"
    season_multipliers: Dict[Season, float] = field(default_factory=lambda: {
        Season.LOW: 0.90,
        Season.SHOULDER: 1.00,
        Season.HIGH: 1.20,
        Season.PEAK: 1.40,
    })
    month_to_season: Dict[int, Season] = field(default_factory=lambda: {
        1: Season.PEAK,  2: Season.HIGH,  3: Season.SHOULDER, 4: Season.LOW,
        5: Season.LOW,   6: Season.HIGH,  7: Season.PEAK,     8: Season.HIGH,
        9: Season.SHOULDER, 10: Season.SHOULDER, 11: Season.HIGH, 12: Season.PEAK,
    })

    def season_for(self, on_date: date) -> Season:
        return self.month_to_season.get(on_date.month, Season.SHOULDER)

    def multiplier_for(self, on_date: date) -> float:
        return self.season_multipliers[self.season_for(on_date)]


In [17]:
# ===========================[ TouristGroup ]========================
@dataclass
class TouristGroup:
    group_name: str
    size: int
    per_person_budget: float
    preferences: List[str] = field(default_factory=list)
    home_country: Optional[str] = None

    def __post_init__(self) -> None:
        if self.size <= 0:
            raise ValidationError("Group size must be positive.")
        if self.per_person_budget <= 0:
            raise ValidationError("Per-person budget must be positive.")


In [18]:
# ===========================[ Experience Hierarchy ]================
class Experience(ABC):
    def __init__(self, name: str, country: Country, base_price: float, capacity: int) -> None:
        self.name = name
        self.country = country
        self.base_price = float(base_price)
        self.capacity = int(capacity)
        self._availability: Dict[date, int] = {}

    # availability
    def set_availability(self, on_date: date, capacity: int) -> None:
        if capacity < 0:
            raise AvailabilityError("Capacity cannot be negative.")
        self._availability[on_date] = min(capacity, self.capacity)

    def available_capacity(self, on_date: date) -> int:
        return self._availability.get(on_date, self.capacity)

    def reserve(self, on_date: date, qty: int) -> None:
        if qty <= 0:
            raise AvailabilityError("Reservation quantity must be positive.")
        remaining = self.available_capacity(on_date)
        if qty > remaining:
            raise AvailabilityError(
                f"{self.name}: requested {qty}, remaining {remaining} on {on_date}."
            )
        self._availability[on_date] = remaining - qty

    # pricing
    def price_for(self, on_date: date) -> float:
        return round(self.base_price * self.country.multiplier_for(on_date), 2)

    @property
    @abstractmethod
    def category(self) -> str:
        ...

class SafariExperience(Experience):
    @property
    def category(self) -> str:
        return "safari"

    def price_for(self, on_date: date) -> float:
        price = super().price_for(on_date)
        if on_date.month in (6, 7, 8):
            price *= 1.10
        return round(price, 2)

class BeachExperience(Experience):
    @property
    def category(self) -> str:
        return "beach"

    def price_for(self, on_date: date) -> float:
        price = super().price_for(on_date)
        if on_date.month in (12, 1):
            price *= 1.08
        return round(price, 2)

class CulturalExperience(Experience):
    @property
    def category(self) -> str:
        return "cultural"

class AdventureExperience(Experience):
    @property
    def category(self) -> str:
        return "adventure"

    def price_for(self, on_date: date) -> float:
        price = super().price_for(on_date)
        if on_date.month in (9, 10):
            price *= 1.05
        return round(price, 2)


In [19]:
# ===========================[ Package & Item ]======================
@dataclass
class PackageItem:
    experience: Experience
    date: date
    nights: int = 1
    discount: float = 0.0

    def price_per_person(self) -> float:
        base = self.experience.price_for(self.date) * max(1, self.nights)
        if self.discount:
            base *= (1 - self.discount)
        return round(base, 2)

@dataclass
class Package:
    name: str
    items: List[PackageItem] = field(default_factory=list)

    def add_item(self, item: PackageItem) -> None:
        self.items.append(item)

    @property
    def countries(self) -> List[str]:
        return list({it.experience.country.name for it in self.items})

    def is_cross_border(self) -> bool:
        return len(self.countries) > 1

    def total_per_person(self) -> float:
        return round(sum(i.price_per_person() for i in self.items), 2)

    def validate(self, group: TouristGroup) -> None:
        total = self.total_per_person()
        if total > group.per_person_budget:
            raise ValidationError(
                f"Per-person total {total} exceeds budget {group.per_person_budget}."
            )
        if group.preferences:
            cats = {it.experience.category for it in self.items}
            if not any(p in cats for p in group.preferences):
                raise ValidationError(
                    f"Package categories {cats} do not match preferences {group.preferences}."
                )
        if self.is_cross_border():
            total_nights = sum(i.nights for i in self.items)
            if total_nights < 2:
                raise ValidationError("Cross-border packages must be at least 2 nights.")


In [20]:
# ===========================[ Booking & Manager ]===================
@dataclass
class Booking:
    package: Package
    group: TouristGroup
    created_at: datetime = field(default_factory=datetime.utcnow)
    status: str = "PENDING"  # PENDING / CONFIRMED / CANCELLED

    def total_cost(self) -> float:
        return round(self.package.total_per_person() * self.group.size, 2)

class BookingManager:
    def __init__(self) -> None:
        self._bookings: List[Booking] = []

    def preview_cost(self, package: Package, group: TouristGroup) -> float:
        package.validate(group)
        return package.total_per_person() * group.size

    def confirm(self, package: Package, group: TouristGroup) -> Booking:
        package.validate(group)
        for item in package.items:
            item.experience.reserve(item.date, group.size)
        booking = Booking(package=package, group=group, status="CONFIRMED")
        self._bookings.append(booking)
        return booking

    def cancel(self, booking: Booking) -> None:
        if booking.status != "CONFIRMED":
            raise BookingError("Only confirmed bookings can be cancelled.")
        for item in booking.package.items:
            current = item.experience.available_capacity(item.date)
            restored = min(item.experience.capacity, current + booking.group.size)
            item.experience.set_availability(item.date, restored)
        booking.status = "CANCELLED"

    def list_bookings(self, status: Optional[str] = None) -> List[Booking]:
        return [b for b in self._bookings if b.status == status] if status else list(self._bookings)


In [21]:
# ===========================[ Breakdown Helper ]===========
def print_price_breakdown(pkg: Package) -> None:
    print("\n— Price Breakdown —")
    for item in pkg.items:
        base = item.experience.base_price
        season = item.experience.country.season_for(item.date)
        country_mult = item.experience.country.multiplier_for(item.date)
        category = item.experience.category
        adjusted = item.experience.price_for(item.date)
        nights = item.nights
        disc = f"{int(item.discount * 100)}%" if item.discount else "none"
        print(f"{item.experience.name} ({category}, {item.experience.country.name})")
        print(f"  Date: {item.date}, Season: {season.name}")
        print(f"  Base: {base:.2f} {item.experience.country.currency}")
        print(f"  Multiplier: ×{country_mult}")
        print(f"  Nights: {nights}, Discount: {disc}")
        print(f"  Final per-person: {adjusted * nights * (1 - item.discount):.2f} {item.experience.country.currency}\n")
